In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import StandardScaler

In [3]:
# Fog Data
fogData = xr.open_dataset('../../../../Data/FogData/CombinedFogData_25Stations.nc')
#plt.figure(figsize=[16,8])
StackFog=fogData.fogdata.stack(a=('years','months','days'))
StackFog
dd =[];
for i in range(StackFog.years.values.shape[0]):
    dd=dd+[str(StackFog.years[i].values)+'-'+str(StackFog.months[i].values)+"-"+str(StackFog.days[i].values)]
fg = xr.Dataset({'fogdata': (('time','stations'), StackFog.values.T)}, coords={'time': pd.to_datetime(dd),'stations': fogData.stations})
y=fg.fogdata.sum(dim='stations').sel(time=slice('1980-12-1','2018-1-31'))
y=y/25.0
y

<xarray.DataArray 'fogdata' (time: 2356)>
array([0.12, 0.12, 0.12, ..., 0.64, 0.28, 0.16])
Coordinates:
  * time     (time) datetime64[ns] 1980-12-01 1980-12-02 ... 2018-01-31

In [11]:
AOData = xr.open_dataset('../../../../Data/AO_EU/AO_Calculated.nc')
AO5D=AOData.AO.rolling(time=5).mean()
aoTS=AO5D
ao=aoTS[((aoTS.time.dt.month>11) | (aoTS.time.dt.month<2)) & (aoTS.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'))
np.corrcoef(y,ao)

array([[ 1.       , -0.0227776],
       [-0.0227776,  1.       ]])

In [20]:
AOData = xr.open_dataset('../../../../Data/AO_EU/AO_Calculated.nc')
AO5D=AOData.AO.rolling(time=60).mean()
aoTS=AO5D
ao=aoTS[((aoTS.time.dt.month>11) | (aoTS.time.dt.month<2)) & (aoTS.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'))
np.corrcoef(y,ao)

array([[ 1.       , -0.0294854],
       [-0.0294854,  1.       ]])

In [23]:
EUData = xr.open_dataset('../../../../Data/AO_EU/EU.nc')
EU5D=EUData.EUVal.rolling(time=30).mean()
euTS=EU5D
eu=euTS[((euTS.time.dt.month>11) | (euTS.time.dt.month<2)) & (euTS.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'))

np.corrcoef(y,eu)


array([[1.       , 0.0456141],
       [0.0456141, 1.       ]])

In [6]:
dataset=xr.open_dataset('VarDates.nc')
dataset
#y

<xarray.Dataset>
Dimensions:  (ncl0: 2356, ncl1: 36, ncl2: 2356)
Dimensions without coordinates: ncl0, ncl1, ncl2
Data variables:
    X        (ncl0, ncl1) float32 ...
    Y        (ncl2) float32 ...

In [7]:
varList=["t2mVal","wind10Val","rh2mVal","invVal","wVal",
         "v_shearVal","u_shearVal","t2mTendVal","mslTendVal",
         "wTendVal","qadvTendVal","z500Val","t2mVal1D","wind10Val1D",
         "rh2mVal1D","invVal1D","wVal1D","v_shearVal1D","u_shearVal1D",
         "t2mTendVal1D","mslTendVal1D","wTendVal1D","qadvTendVal1D","z500Val1D","t2mVal2D",
         "wind10Val2D","rh2mVal2D","invVal2D","wVal2D","v_shearVal2D",
         "u_shearVal2D","t2mTendVal2D","mslTendVal2D","wTendVal2D",
         "qadvTendVal2D","z500Val2D"]

In [8]:
#ff=dataset.Y
#ff.assign_coords(ncl2=y.time)

ff=xr.Dataset({'ff': (('time'), dataset.Y.values)}, coords={'time': y.time})
#ff

Xs=xr.Dataset({'x': (('time','vars'), dataset.X.values)}, coords={'time': y.time,'vars': varList})
Xs

<xarray.Dataset>
Dimensions:  (time: 2356, vars: 36)
Coordinates:
  * time     (time) datetime64[ns] 1980-12-01 1980-12-02 ... 2018-01-31
  * vars     (vars) <U13 't2mVal' 'wind10Val' ... 'qadvTendVal2D' 'z500Val2D'
Data variables:
    x        (time, vars) float32 1.72 -0.3298 -0.4251 ... 0.9548 -0.6485 1.499

In [9]:
CC=xr.corr(ff.ff,Xs.x,dim='time')
for i in CC:
    print(i.values)



-0.2900602986874547
-0.27634472583498443
0.5799269853562218
-0.19760134876898336
-0.0012193186290752193
0.05494627134416614
-0.1372561694505029
0.03674463955769025
0.016350198059048252
0.047752995451885355
0.007756261674234085
0.12852109855079413
-0.2999442410955356
-0.38331406013371266
0.5891916939642677
-0.23542328671919777
-0.05012464785215953
0.04613451013774542
-0.14717420247533297
0.010914953853133496
0.05692974547240416
0.020437140859841198
0.005150044844793475
0.14674479119279488
-0.3001419300488823
-0.27869133602660623
0.546711598989154
-0.2782986656524827
-0.06724815915647518
0.0015269282053364816
-0.12386347725768333
0.010211839190156146
-0.009656126015123518
-0.019835150645735645
0.012335212334476943
0.15824415299628108
